 - train.csv - the training set
 - test.csv - the test set
 - data_description.txt - full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here
 - sample_submission.csv - a benchmark submission from a linear regression on year and month of sale, lot square footage, and number of bedrooms
 
[kaggle_datasets](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques)

Задача предсказания цен на недвижимость с Kaggle

### ✔ Исследуем данные

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np

#  записываем csv-файл в объект DataFrame
path = r'C:\Users\79181\n.brykovskaya\module_2\4ML\5ML_project\files\train.csv'
train = pd.read_csv(path)

# выводим первые 5 наблюдений
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
# смотрим форму обучающего датафрейма
train.shape


(1460, 81)

### ✔ Удаление зависимой переменной из обучающего набора
Зависимой переменной является SalesPrice,которую мы превратим в массив меток. Он будет использован позднее при обучении модели.

In [3]:
# создаем массив меток
y = train.pop('SalePrice').values

### ✔  Кодировка отдельного столбца со строковыми значениями
Для начала закодируем отдельный столбец со строковыми значениями HouseStyle, который содержит значения, описывающие внешний облик дома. Давайте выведем частоту каждого строкового значения.

In [4]:
# выводим частоты каьтегорий переменной HouseStyle
vc = train['HouseStyle'].value_counts()
vc

HouseStyle
1Story    726
2Story    445
1.5Fin    154
SLvl       65
SFoyer     37
1.5Unf     14
2.5Unf     11
2.5Fin      8
Name: count, dtype: int64

Данный столбец содержит 8 уникальных значений.

Scikit-Learn требует чтобы данными были строго двумерные. Можно заставить библиотеку Pandas  создать объект dataframe  с единственным столбцом, передав список, состоящий из одного элемента, в квадратные скобки.

In [5]:
# создаем объект DataFrame с одним столбцом
hs_train = train[['HouseStyle']].copy()
hs_train.ndim

2

### ✔ Импортируем класс, создаем экземпляр класса - модель, обучаем модель - трехэтапный процесс работы с моделью

Импортируем класс `OneHotEncoder`, создаем экземпляр класса, убеждаемся в том, что мы получили плотный (а не разреженный) массив, наконец, выполняем дамми-кодирование нашего отдельного столбца с помощью метода `.fit_transform()`

In [6]:
# Импортируем класс OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
# создаем экземпляр класса OneHotEncoder,
# будем возвращать плотный массив
ohe = OneHotEncoder(sparse_output=False)
# выполняем дамми-кодирование переменной
# HouseStyle в обучающем наборе
hs_train_tranceformed = ohe.fit_transform(hs_train)
# смотрим результат 
hs_train_tranceformed 

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

как и ожидалось, каждое уникальное значение становится отдельным бинарным признаком

Убеждаемся, что каждое уникальное значение получило свой столбец (8 уникальных значений дадут 8 столбцов).

In [7]:
hs_train_tranceformed.shape

(1460, 8)

### ✔ Извлекаем имена столбцов
Полученный объект - это массив NumPy, а не датафрейм pandas. Извлечем имя столбца из объекта  OneHotEncoder с помощью  метода `метод get_feature_names_out()`

In [8]:
# запишем имена столбцов в объект feature_names
feature_names = ohe.get_feature_names_out()
# смотрим имена столбцов
feature_names

array(['HouseStyle_1.5Fin', 'HouseStyle_1.5Unf', 'HouseStyle_1Story',
       'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf', 'HouseStyle_2Story',
       'HouseStyle_SFoyer', 'HouseStyle_SLvl'], dtype=object)

### ✔ Проверка корректности первой строки данных

In [9]:
# выведем первую строку преобразованных данных
row0 = hs_train_tranceformed[0]
row0

array([0., 0., 0., 0., 0., 1., 0., 0.])

Шестое значение массива было закодировано единицей. Используя логическое индексирование, можно выяснить имя переменной

In [10]:
# выясним имя переменной к-е в 1й строке закодировано как1
feature_names[row0==1]

array(['HouseStyle_2Story'], dtype=object)

Теперь убедимся, что первое значение в нашем исходном столбце является тем же самым

In [11]:
# смотрим значение исходного столбца HouseStyle в строке 0
hs_train.values[0]

array(['2Story'], dtype=object)

### ✔ Метод .inverse_transform() для автоматизации
У объекта OneHotEncoder есть метод `.inverse_transform()`, который вернет нам исходные данные. Здесь мы должны обернуть row0 в список, чтобы получить двумерный массив

In [12]:
# выполним обратное преобразование
ohe.inverse_transform([row0])

array([['2Story']], dtype=object)

Мы можем проверить все значения сразу, выполнив инвертирование всегопреобразованного массива

In [13]:
# выполняем инвертирование всего преобразованного массива
hs_inv = ohe.inverse_transform(hs_train_tranceformed)
hs_inv

array([['2Story'],
       ['1Story'],
       ['2Story'],
       ...,
       ['2Story'],
       ['1Story'],
       ['1Story']], dtype=object)

In [14]:
# проверяем совпадают ли массивы
np.array_equal(hs_inv, hs_train.values)

True

### ✔ Применение преобразования к тестовому набору
Любое преобразование, применяемое к обучающему набору, должно быть применено и к тестовому набору. Прочитаем тестовый набор данных, извлечем тот же самый столбец и применим к нему наше преобразование

In [15]:
# выполним дамми-кодирование переменной HouseStyle в тестовом наборе
#  записываем csv-файл в объект DataFrame
path_t = r'C:\Users\79181\n.brykovskaya\module_2\4ML\5ML_project\files\test.csv'
test = pd.read_csv(path_t)
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [16]:
# выводим частоты категорий переменной HouseStyle
vc_t = test['HouseStyle'].value_counts()
vc_t

HouseStyle
1Story    745
2Story    427
1.5Fin    160
SLvl       63
SFoyer     46
2.5Unf     13
1.5Unf      5
Name: count, dtype: int64

In [17]:
# создаем объект DataFrame с одним столбцом
hs_test = test[['HouseStyle']].copy()

# выполняем дамми-кодирование переменной
# HouseStyle в тестовом наборе
hs_test_tranceformed = ohe.fit_transform(hs_test)
# смотрим результат 
hs_test_tranceformed 

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

По итогам дамми-кодирования должно получиться 7 столбцов, что мы и видим. Убеждаемся, что каждое уникальное значение получило свой столбец (7 уникальных значений дадут 7 столбцов).

В тестовом наборе уникальных значений на 1 меньше, чем в обучающем.

In [18]:
hs_test_tranceformed.shape

(1459, 7)

### ✔ Применение конвейера
В библиотеке scikit-learn есть класс Pipeline. который принимает список преобразований и выполняет их последовательно. Кроме того, в него можно поместить модель машинного обучения в качестве конечного класса.

Приведем пример простой импутации и дамми-кодирования


In [19]:
# импортируем класс Pipeline
from sklearn.pipeline import Pipeline
# импортируем класс SimpleImputer
from sklearn.impute import SimpleImputer

Каждый этап конвейера представляет собой кортеж, состоящий из двух элементов - названия шага и экземпляра класса.  Результат предыдущего этапа будет входными данными для последующего этапа.


In [20]:
# задаем этап имутации
si_step =('si', SimpleImputer(strategy='constant',
                             fill_value='MISSING'))
# задаем этап дамми кодирования
ohe_step = ('ohe', OneHotEncoder(sparse_output=False,
                                handle_unknown='ignore'))

In [21]:
# создаем список этапов
steps = [si_step, ohe_step]
# создаем конвейер, передавая список этапов
pipe = Pipeline(steps)

In [22]:
# копируем переменную HouseStyle в обучающем наборе
hs_train = train[['HouseStyle']].copy()
# заменяем первое значение на пропуск
hs_train.iloc[0,0] = np.nan

Выполняем импутацию и дамми-кодирование переменной HouseStyle в обучающем наборе с помощью каждого этапа конвейера и убеждаемся, что каждое уникальное значение получило свой столбец (8 уникальных значений плюс 1 новое значение MISSING дадут 9 столбцов)

In [23]:
hs_transformed = pipe.fit_transform(hs_train)
hs_transformed.shape

(1460, 9)

Преобразование для тестового набора легко выполняются с помощью соответсвующего этапа конвейера, для этого нужно просто передать тестовый набор в метод `.transform()`

In [24]:
# копируем переменную HouseStyle в тестовом наборе
hs_test = test[['HouseStyle']].copy()

Выполняем импутацию и дамми-кодирование переменной HouseStyle в контрольном наборе с помощью каждого этапа конвейера и убеждаемся, что каждое уникальное значение получило свой столбец (7 уникальных значений плюс 1 новое значение MISSING и 1 значение из обучающего набора, которого нет в тестовом дадут 9 столбцов)

In [25]:
hs_test_transformed = pipe.transform(hs_test)
hs_test_transformed.shape

(1459, 9)

Для выполнения преобразования тестового набора нам нужно вызвать метод `.transform()` а не `.fit_transform()` поскольку библиотека scikit-learn уже нашла всю необходимую информацию, которая ей понадобится для преобразования любого другого набора, содержащего те же самые имена столбцов.

### ✔ Применение преобразований для нескольких столбцов со строковыми значениями.

Для преобразования нескольких столбцов со строковыми значениями также просто. необходимо выбрать нужные столбцы и передать их в тот же самый конвейер.

In [26]:
# создадим список столбцов RoofMatl и HouseStyle со строковыми значениями
string_cols = ['RoofMatl', 'HouseStyle']
# создаем датафрейм из двух столбцов 'RoofMatl', 'HouseStyle'
string_train = train[string_cols]
# выводим первые 3 наблюдения
string_train.head(3)

,RoofMatl,HouseStyle
0,CompShg,2Story
1,CompShg,1Story
2,CompShg,2Story


### ✔ Обращение к отдельным этапам конвейера
С помощью атрибута named_steps можно обратиться к отдельному этапу конвейера по его имени.

Обратимся к OneHotEncoder, чтобы вывести названия признаков


In [27]:
# обращение к этапу ohe
ohe = pipe.named_steps['ohe']
# извлекаем названия признаков
ohe.get_feature_names_out()

array(['x0_1.5Fin', 'x0_1.5Unf', 'x0_1Story', 'x0_2.5Fin', 'x0_2.5Unf',
       'x0_2Story', 'x0_MISSING', 'x0_SFoyer', 'x0_SLvl'], dtype=object)

### ✔ Использование нового ColumnTransformer для отбора столбцов
Класс ColumnTransformer позволяет определить, для каких столбцов нужно выполнить определенные преобразования. Категориальные признаки мы почти всегда преобразовываем иначе, чем количественные.

Выполним ту же самую импутацию и дамми-кодирование, что и в предыдущих примерах, но теперь с использованием ColumnTransformer. При этом конвейер остается тем же самым, что и прежде, просто к имени каждого этапа добавим префикс cat

In [28]:
# импортируем класс ColumnTransformer
from sklearn.compose import ColumnTransformer

# задаем этап имутации
cat_si_step =('si', SimpleImputer(strategy='constant',
                             fill_value='MISSING'))
# задаем этап дамми кодирования
cat_ohe_step = ('ohe', OneHotEncoder(sparse_output=False,
                                handle_unknown='ignore'))
# создаем список этапов
cat_steps = [cat_si_step, cat_ohe_step]
# создаем конвейер, передавая список этапов
cat_pipe = Pipeline(cat_steps)
# создадим список столбцов RoofMatl и HouseStyle со строковыми значениями
cat_cols = ['RoofMatl', 'HouseStyle']
# создаем список с трехэлементным кортежем
cat_transformers = [('cat', cat_pipe, cat_cols)]
# передаем список в  ColumnTransformer
ct = ColumnTransformer(transformers=cat_transformers)


#### ✔ Передаем весь объект DataFrame  в  ColumnTransformer
Экземпляр класса ColumnTransformer отбирает нужные нам столбцы, поэтому мы можем просто передать весь датафрейм в метод `.fit_transforme()` Нужные столбцы будут выбраны автоматически.

In [29]:
# передаем обучающий набор в ColumnTransformer
X_cat_transformed= ct.fit_transform(train)
# убеждаемся, что преобразования выполнены
X_cat_transformed.shape

(1460, 16)

In [30]:
# передаем тестовый набор в ColumnTransformer
X_cat_transformed_test  = ct.transform(test)
# убеждаемся, что преобразования выполнены
X_cat_transformed_test.shape

(1459, 16)

#### ✔ Извлечение названий признаков.
Все классы, выполняющие преобразования, хранятся в атрибует `named_transeformers_`. После этого извлекаем объект OneHotEncoder, и уже из него можно извлечь имена предикторов

In [31]:
# выбираем трансформер
pl = ct.named_transformers_['cat']
# извлекаем объект OneHotEncoder
ohe = pl.named_steps['ohe']
# извлекаем имена признаков
ohe.get_feature_names_out()

array(['x0_ClyTile', 'x0_CompShg', 'x0_Membran', 'x0_Metal', 'x0_Roll',
       'x0_Tar&Grv', 'x0_WdShake', 'x0_WdShngl', 'x1_1.5Fin', 'x1_1.5Unf',
       'x1_1Story', 'x1_2.5Fin', 'x1_2.5Unf', 'x1_2Story', 'x1_SFoyer',
       'x1_SLvl'], dtype=object)

### ✔ Преобразование количественных переменных
Количественным переменным могут потребоваться совершеноно другие преобразования. Вместо импутации пропусков константами чаще используется импутация средним значением или медианой. И вместо дамм-кодирования обычно выполняют стандартизацию путем вычитания из исходного значения признака среднего значения и делением полученного результата на стандартное отклонение. Это позволяет улучшать качество регрессионных моделей.

### ✔ Работа со всеми количественными признаками
Вместо выбора одного или нескольких столбцов вручную, как это делали ранее работая со столбцами, содержащими строковые значения, можно выбрать все количественные переменные. 

Чтобы это сделать, нужно сначала найти тип каждого признакас помощью атрибута `dtypes`, после чего проверить, имеет ли данный признак тип` object`.



In [32]:
# смотрим типы первых 5 признаков
train.dtypes.head()

Id               int64
MSSubClass       int64
MSZoning        object
LotFrontage    float64
LotArea          int64
dtype: object

C помощью атрибута kind выводим тип каждой переменной в виде одной буквы


In [33]:
kinds = np.array([dt.kind for dt in train.dtypes])
kinds[:5]

array(['i', 'i', 'O', 'f', 'i'], dtype='<U1')

Предположим, что все количественные предикторы не относятся к типу object. Исходя из этого, мы можем извлечь категориальные признаки

In [34]:
# создаем массив из всех признаков
all_columns = train.columns.values
# создаем массив из количественных признаков
is_num = kinds != 'O'
# отбираем только количественные признаки
num_cols = all_columns[is_num]
# выводим первые 5 колич.признаков
num_cols[:5]

array(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual'],
      dtype=object)

In [35]:
# отбираем только категориальные признаки
cat_cols = all_columns[~is_num]
# выводим первые 5 категор.признаков
cat_cols[:5]

array(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour'],
      dtype=object)

Раз количественные признаки определены, то мы сможем вновь воспользоваться классом ColumnTransformer

In [36]:
# импортируем класс StandartScaler
from sklearn.preprocessing import StandardScaler
# задаем этап импутации
nun_si_step = ('si', SimpleImputer(strategy='median'))
# задаем этап стандартизации
scaler = StandardScaler()
num_ss_step = ('ss', scaler)
# задаем список этапов
num_steps = [nun_si_step, num_ss_step]
# создаем конвейер, передав список этапов
num_pipe = Pipeline(num_steps)
# создаем список с трехэлементным кортежем
num_transformers = [('num', num_pipe, num_cols)]
# создаем трансформер, передав список
ct = ColumnTransformer(transformers=num_transformers)
# выполняем преобразования обучающего набора с помощью трансформера
X_num_transformed = ct.fit_transform(train)
# убеждаемся, что преобразования выполнены
X_num_transformed.shape

(1460, 37)

### ✔ Передаем конвейер с преобразованиями для категориальных признаков и конвейер с преобразованиями для количественных признаков в  ColumnTransformer

С помощью ColumnTransformer для каждого типа столбцов можно применить отдельные преобразования. 

Нам нужно определиться с операциями преобразования для категориальных признаков и операциями преобразования для количественных признаков. 

После этого создаем отдельный конвейер для категориальных признаков, отдельный конвейер для количественных признаков, затем используем ColumnTransformer для независимого преобразования признаков. 

Преобразования в обеих конвейерах будут выполняться параллельно, а их результаты будут сконкатенированы.

Создаем список трехэлементных кортежей, в котором первый элемент кортежа - название конвейера с преобразованиями для определенного типа признаков

In [37]:
transformers =[('cat', cat_pipe, cat_cols),
              ('num', num_pipe, num_cols)
              ]

# передаем список в ColumnTransformer
ct = ColumnTransformer(transformers=transformers)
# выполняем преобразование для категориальных т колич. признаков
X = ct.fit_transform(train)
# убежджаемся, что преобразование выполнено
X.shape

(1460, 304)

### ✔ Машинное обучение
Целью всех этапов была подготовка данных для последующего машинного обучения. Мы можем создать итоговый конвейер и добавить в него в качестве последнего этапа модель машинного обучения.

Первым этапом этого конвейера будет преобразование данных. описанных выше.

В качестве зависимой переменной `y`, как было принято выше, берется переменная `SalePrice`. Здесь мы просто используем метод `.fit()` вместо `.fit_transform()`, поскольку итоговым этапом конвейера будет модель машинного обучения, которая не производит ниаких преобразований.


In [38]:
# импортируем класс Ridge
from sklearn.linear_model import Ridge
# добавляем в конвейер новый этап - модель машинного обучения
# модель гребневой регрессии
ml_pipe = Pipeline([('transform', ct), ('ridge', Ridge())])
# выполняем преобразования и обучаем модель гребневой регрессии
ml_pipe.fit(train, y)

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('si',
                                                                   SimpleImputer(fill_value='MISSING',
                                                                                 strategy='constant')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  array(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition...
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'], dtype=object))])),
                ('ridge', Ridge())])

Оценить качество модели можно, например, с помощью метода `.score()`, возвращающего R-квадрат

In [39]:
# оцениваем качество модели
ml_pipe.score(train, y)

0.9220446277437464

### ✔ Перекрестная проверка.
Разумеется, оценка модели на обучающем наборе данных бесполезна. Чтобы лучше понять, как модель поведет себя на неизвестных ей данных, нужно провести k-блочную перекрестную проверку. Для производимости результатов нужно задать значение random_state

In [40]:
# импортируем класс KFold функцию cross_val_score()
from sklearn.model_selection import KFold, cross_val_score
# создадим экземпляр класса KFold
kf = KFold(n_splits=5, shuffle=True, random_state=123)
# выполняем перекрестную проверку, конвейер размещен внутри цикла перекрестной проверки
cross_val_score(ml_pipe, train, y, cv=kf).mean()

0.8135802110436661

### ✔ Отбор наилучших значений гиперпараметров с помощью решетчатого поиска
Решетчатый поиск в библиотеке scikit-learn требует передачи словаря, ключами которого будут названия гиперпараметров, а значениями - списки значений этих гиперпараметров.

При работе с конвейером мы должны взять имя этапа с двумя символами нижнего подчеркивания на концк и после этого указать имя гиперпараметра. Если мы работаем с многоуровневым конвейером, как в примере ниже, то тогда двойное нижнее подчеркивание должно разделять имена всех уровней (transform__num__si__strategy, здесь мы работаем с уровнями transform, num и si), пока не будет достигнуто название гиперпараметра экземпляра класса (в данном случае название гиперпараметра strategy, объект si, экземпляр класса SimpleImputer), по значениям которого должен быть осуществлен поиск (в данном случае, как можно увидеть ниже, поиск должен быть осуществлен по значениям 'mean' и 'median')

In [41]:
# импортируем класс GridSearchCV
from sklearn.model_selection import GridSearchCV
# задаем сетку гиперпараметров
param_grid = {
    'transform__num__si__strategy': ['mean', 'median'],
    'ridge__alpha': [.001, 0.1, 1.0, 5, 10, 50, 100, 1000]
}

# передаем конвейер в объект GridSearchCV
gs = GridSearchCV(ml_pipe, param_grid, cv=kf, return_train_score=True)
# выполняем решетчатый поиск
gs.fit(train, y)
# смотрим наилучшие значения гиперпараметров
gs.best_params_

{'ridge__alpha': 10, 'transform__num__si__strategy': 'median'}

In [42]:
# смотрим наилучшее значение R-квадрат
gs.best_score_

0.8191174047000169

### ✔ Представление результатов решетчатого поиска в виде датафрейма pandas
Результаты решетчатого поиска хранятся в атрибуте `cv_results_`. Он представляет собой словарь, а значит, его можно представить в виде объекта DataFrame для удобства чтения


In [43]:
# представляем результаты решетчатого поиска в виде DataFrame 
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ridge__alpha,param_transform__num__si__strategy,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.054004,0.001898,0.019803,0.000748,0.001,mean,"{'ridge__alpha': 0.001, 'transform__num__si__s...",0.898238,0.023654,0.775710,...,0.649027,0.323896,16,0.935221,0.945615,0.945853,0.938883,0.933089,0.939732,0.005240
1,0.056003,0.001675,0.019199,0.000749,0.001,median,"{'ridge__alpha': 0.001, 'transform__num__si__s...",0.898343,0.023932,0.775815,...,0.649122,0.323821,15,0.935215,0.945617,0.945866,0.938902,0.933146,0.939749,0.005229
2,0.057801,0.002788,0.020799,0.000980,0.100,mean,"{'ridge__alpha': 0.1, 'transform__num__si__str...",0.897427,0.779122,0.800158,...,0.803011,0.087572,12,0.934177,0.942729,0.944076,0.937618,0.932917,0.938303,0.004459
3,0.055602,0.000800,0.019397,0.000485,0.100,median,"{'ridge__alpha': 0.1, 'transform__num__si__str...",0.897528,0.779363,0.800283,...,0.803135,0.087563,11,0.934170,0.942731,0.944089,0.937636,0.932974,0.938320,0.004450
4,0.052806,0.001939,0.019796,0.000750,1.000,mean,"{'ridge__alpha': 1.0, 'transform__num__si__str...",0.892205,0.809303,0.822060,...,0.813494,0.079651,8,0.922729,0.931674,0.928383,0.925246,0.929364,0.927479,0.003147
5,0.052404,0.002724,0.019195,0.000402,1.000,median,"{'ridge__alpha': 1.0, 'transform__num__si__str...",0.892255,0.809387,0.822202,...,0.813580,0.079648,7,0.922720,0.931680,0.928386,0.925256,0.929416,0.927492,0.003156
6,0.049402,0.000490,0.018599,0.000488,5.000,mean,"{'ridge__alpha': 5, 'transform__num__si__strat...",0.890080,0.820475,0.820297,...,0.817483,0.071797,4,0.906931,0.916927,0.912648,0.909260,0.920303,0.913214,0.004888
7,0.051207,0.000749,0.018994,0.000002,5.000,median,"{'ridge__alpha': 5, 'transform__num__si__strat...",0.890110,0.820468,0.820432,...,0.817530,0.071819,3,0.906927,0.916943,0.912645,0.909265,0.920346,0.913225,0.004903
8,0.048603,0.000490,0.018797,0.000400,10.000,mean,"{'ridge__alpha': 10, 'transform__num__si__stra...",0.890602,0.824404,0.819722,...,0.819080,0.070268,2,0.899408,0.909891,0.906001,0.901843,0.915029,0.906434,0.005594
9,0.051799,0.001165,0.019201,0.000979,10.000,median,"{'ridge__alpha': 10, 'transform__num__si__stra...",0.890652,0.824369,0.819853,...,0.819117,0.070311,1,0.899404,0.909909,0.905997,0.901847,0.915072,0.906446,0.005610
